<img src="images/stable_marriage.png" align=right width=30%>
# College Admissions and the Stability of Marriage
Author: Jin Yeom (jinyeom@utexas.edu)

## Contents
- [Stable marriage problem](#Stable-marriage-problem)
- [College admission problem](#College-admission-problem)
- [References](#References)

In [23]:
import random
from pprint import pprint
from copy import deepcopy
from typing import Mapping, Sequence, Tuple
from collections import defaultdict

In this notebook, we'll solve a little problem my girlfriend's sorority is having: matching new littles and bigs. Fortunately, the same problem has been solved long time ago by mathematicians named David Gale and Lloyd Shapley. They called this problem the **stable marriage problem**, or the **college admissions problem** for more general cases.

## Stable marriage problem

*Given n men and n women, where each person has ranked all members of the opposite sex in order of preference, marry the men and women together such that there are no two people of opposite sex who would both rather have each other than their current partners. When there are no such pairs of people, the set of marriages is deemed stable.*

In [14]:
# Example from https://en.wikipedia.org/wiki/Stable_marriage_problem
# The preferences are modified to make the problem a little more challenging.
suiter_prefs = {"A": ["Y", "X", "Z"],
                "B": ["Y", "Z", "X"],
                "C": ["X", "Z", "Y"]}

reviewer_prefs = {"X": ["B", "A", "C"],
                  "Y": ["C", "B", "A"],
                  "Z": ["C", "A", "B"]}

In [15]:
def gale_shapley(suiter_prefs: Mapping[str, Sequence[str]], 
                 reviewer_prefs: Mapping[str, Sequence[str]]) -> Mapping[str, str]:
    suiters = list(suiter_prefs.keys())
    reviewers = list(reviewer_prefs.keys())
    # NOTE: this may seem wasteful, but it's either this or linear-time
    # search over suiters within each loop; we wouldn't want that, would we?
    suiter_matches = dict([(s, None) for s in suiters])
    reviewer_matches = dict([(r, None) for r in reviewers])
    while suiters:
        suiter = suiters.pop(0)
        reviewer = suiter_prefs[suiter][0]
        # if the current reviewer is not matched yet, match them
        if reviewer not in suiter_matches.values():
            suiter_matches[suiter] = reviewer
            reviewer_matches[reviewer] = suiter
        # otherwise, compare current matches of the two with each other
        else:
            suiter_ = reviewer_matches[reviewer]
            if reviewer_prefs[reviewer].index(suiter) < reviewer_prefs[reviewer].index(suiter_):
                suiter_matches[suiter_] = None
                suiter_matches[suiter] = reviewer
                reviewer_matches[reviewer] = suiter
                suiters.append(suiter_)
            else:
                suiter_prefs[suiter].remove(reviewer)
                suiters.append(suiter)
    return suiter_matches

pprint(gale_shapley(suiter_prefs, reviewer_prefs))

{'A': 'X', 'B': 'Y', 'C': 'Z'}


Now, let's test the algorithm with a larger problem size.

In [16]:
suiters = ["Abigail", "Beth", "Chloe", "Daisy"]
reviewers = ["Emily", "Faith", "Gaby", "Haley"]
suiter_prefs = dict([(s, random.sample(reviewers, len(reviewers))) for s in suiters])
reviewer_prefs = dict([(r, random.sample(suiters, len(suiters))) for r in reviewers])

pprint(suiter_prefs)
pprint(reviewer_prefs)

{'Abigail': ['Emily', 'Gaby', 'Faith', 'Haley'],
 'Beth': ['Faith', 'Gaby', 'Emily', 'Haley'],
 'Chloe': ['Faith', 'Haley', 'Gaby', 'Emily'],
 'Daisy': ['Gaby', 'Faith', 'Emily', 'Haley']}
{'Emily': ['Abigail', 'Daisy', 'Chloe', 'Beth'],
 'Faith': ['Beth', 'Chloe', 'Abigail', 'Daisy'],
 'Gaby': ['Beth', 'Chloe', 'Abigail', 'Daisy'],
 'Haley': ['Daisy', 'Beth', 'Abigail', 'Chloe']}


In [17]:
pprint(gale_shapley(suiter_prefs, reviewer_prefs))

{'Abigail': 'Emily', 'Beth': 'Faith', 'Chloe': 'Haley', 'Daisy': 'Gaby'}


## College admissions problem

~~College admissions problem is a more general variation of stable marriage problem~~ (this statement is proven to be not necessarily true), in which "suiters" and "reviewers" do not have to be of the same numbers, and each suiter (or reviewer) is not required to have all reviewers ranked in its preferences (and vice versa). Each reviwer also has a capacity of how many suiters it can be matched with at a time.

In [25]:
def prefs(A: Sequence[str], 
          B: Sequence[str], 
          size: int) -> Tuple[Mapping[str, Sequence[str]]]:
    r"""Generate a tuple of random preferences of given size for each other"""
    a_prefs = dict([(a, random.sample(B, size)) for a in A])
    b_prefs = dict([(b, random.sample(A, size)) for b in B])
    return a_prefs, b_prefs

In [26]:
suiter_prefs, reviewer_prefs = prefs(suiters, reviewers, 2)
pprint(suiter_prefs)
pprint(reviewer_prefs)

{'Abigail': ['Gaby', 'Emily'],
 'Beth': ['Faith', 'Emily'],
 'Chloe': ['Haley', 'Emily'],
 'Daisy': ['Gaby', 'Faith']}
{'Emily': ['Abigail', 'Beth'],
 'Faith': ['Beth', 'Abigail'],
 'Gaby': ['Daisy', 'Chloe'],
 'Haley': ['Chloe', 'Daisy']}


In [ ]:
def galeshapley

## References

- D. Gale and L. S. Shapley, "College Admissions and the Stability of Marriage", The American Mathematical Monthly, Vol. 69, No. 1 (Jan., 1962), pp. 9-15, Mathematical Association of America
- Roth, A. E. "The College Admissions Problem Is Not Equivalent to the Marriage Problem." Journal of Economic Theory 36 (August 1985): 277–288.